<a href="https://colab.research.google.com/github/Sohammhatre10/Predicto/blob/main/NewsAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.6 MB/s eta 0:00:00


In [2]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at human-centered-summarization/financial-summarization-pegasus and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:

from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

In [4]:
monitored_ticker = ['Bitcoin', 'Dogecoin', 'Litecoin', 'Ethereum', 'XRP']

In [5]:
def search_for_crypto_news(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&sca_esv=554867176&rlz=1C1ONGR_enIN1063IN1063&tbs=sbd:1,qdr:d&tbm=nws&sxsrf=AB5stBjN5-syjYtVHcPU9P30_SJyTV9fUg:1691523802642&source=lnt&sa=X&ved=2ahUKEwi7s4qb6c2AAxUmpVYBHfCNBSoQpwV6BAgBEAg&biw=1280&bih=595&dpr=1.5".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs

In [6]:
raw_urls = {}
for ticker in monitored_ticker:
  raw_urls[ticker] = search_for_crypto_news(ticker)

In [7]:
import re

In [8]:
exclude_list = ['maps', 'policies','preferences', 'accounts', 'support']

In [9]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [10]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_ticker}
cleaned_urls

{'Bitcoin': ['https://finance.yahoo.com/news/opportunity-palantir-technologies-inc-nyse-110015683.html',
  'https://cryptosaurus.tech/the-next-cryptocurrencies-to-explode-on-sunday-august-13th-arwave-algorand-solana/',
  'https://finance.yahoo.com/news/more-bitcoin-etf-decisions-already-140000373.html',
  'https://finance.yahoo.com/news/free-press-withers-el-salvador-040745716.html',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BBitcoin%26sca_esv%3D554867176%26rlz%3D1C1ONGR_enIN1063IN1063%26tbs%3Dsbd:1,qdr:d%26tbm%3Dnws%26sxsrf%3DAB5stBjN5-syjYtVHcPU9P30_SJyTV9fUg:1691523802642%26source%3Dlnt%26sa%3DX%26ved%3D2ahUKEwi7s4qb6c2AAxUmpVYBHfCNBSoQpwV6BAgBEAg%26biw%3D1280%26bih%3D595%26dpr%3D1.5%26pccc%3D1',
  'https://finance.yahoo.com/news/ensuring-peace-mind-cybernetics-safeguards-193922656.html',
  'https://finance.yahoo.com/news/chatgpt-given-20k-invest-stocks-155928839.html',
  'https://theglobalherald.com/business/finance/what-is-the-lk-99-superconductor-plus-amsc-ceo-breaks-

In [11]:
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [12]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_ticker}
articles

{'Bitcoin': ["Using the 2 Stage Free Cash Flow to Equity, Palantir Technologies fair value estimate is US$26.70 Current share price of US$15.41 suggests Palantir Technologies is potentially 42% undervalued  The US$13.83 analyst price target for PLTR is 48% less than our estimate of fair value Today we'll do a simple run through of a valuation method used to estimate the attractiveness of Palantir Technologies Inc. (NYSE:PLTR) as an investment opportunity by estimating the company's future cash flows and discounting them to their present value. The Discounted Cash Flow (DCF) model is the tool we will apply to do this. Before you think you won't be able to understand it, just read on! It's actually much less complex than you'd imagine. Remember though, that there are many ways to estimate a company's value, and a DCF is just one method. For those who are keen learners of equity analysis, the Simply Wall St analysis model here may be something of interest to you.  See our latest analysis 

In [13]:
btc = articles["Bitcoin"]
eth = articles["Ethereum"]
lte = articles["Litecoin"]
xrp = articles["XRP"]
doge = articles["Dogecoin"]

In [14]:
def summarize(articles):
    summaries = []
    for ticker, article_list in articles.items():
        article_text = '\n\n'.join(article_list)  # Join the list of articles into a single string
        ticker_summaries = []
        for chunk in article_text.split('\n\n'):  # Split the article into paragraphs (you can adjust this based on your article structure)
            input_ids = tokenizer.encode(chunk, return_tensors='pt', max_length=512, truncation=True)
            output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
            summary = tokenizer.decode(output[0], skip_special_tokens=True)
            ticker_summaries.append(summary)
        summaries.append({ticker: ticker_summaries})
    return summaries

# Assuming you have a dictionary with lists of articles for each ticker (e.g., btc, eth, doge, xrp, lte)
articles = {
    'BTC': btc,
    'ETH': eth,
    'DOGE': doge,
    'XRP': xrp,
    'LTC': lte
}

# Call the summarize function with the articles dictionary
summaries = summarize(articles)
print(summaries)

[{'BTC': ['View our latest analysis for Palantir Technologies.', 'All photographs subject to copyright.', 'Bitwise’s application for an ETP rejected by SEC. Decisions for applications from BlackRock, VanEck, WisdomTree and Invesco are due', 'Social media influencers are a key part of President Nayib Bukele’s campaign.', 'All images are copyrighted.', 'DropCatch DropCatch DropCatch DropCatch DropCatch DropCatch DropCatch DropCatch DropCatch DropCatch DropCatch DropCatch DropCatch DropCatch has launched a revolutionary solution to restore stolen crypto.', 'NewsBTC conducted an experiment with ChatGPT to invest $20,000. Meta, Rivian and Solana had the highest returns', 'American Superconductor is commercialization of LK-99. McGahn says, ‘We need to be able to develop the future but deliver the present’', 'Follow us on Instagram, Snapchat and Twitter for the latest news.', '.', 'All comments are moderated.', 'All photographs subject to copyright.', 'Onschain’s platform aims at building an 

In [15]:
from transformers import pipeline
import pandas as pd


# Initialize the sentiment analysis pipeline
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model_revision = "af0f99b"
sentiment_classifier = pipeline('sentiment-analysis', model=model_name, revision=model_revision)

def analyze_sentiment(summaries):
    sentiment_results = sentiment_classifier(summaries)  # Use the pipeline directly on the list of summaries
    return sentiment_results

# Assuming you want to analyze the sentiment for the 'BTC' ticker's summaries
btc_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('BTC', [])]
eth_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('ETH', [])]  # List of summaries for ETH
doge_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('DOGE', [])]  # List of summaries for DOGE
xrp_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('XRP', [])]  # List of summaries for XRP
lte_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('LTC', [])]  # List of summaries for LTE

sentiment_results_btc = analyze_sentiment(btc_summaries)
sentiment_results_eth = analyze_sentiment(eth_summaries)
sentiment_results_doge = analyze_sentiment(doge_summaries)
sentiment_results_xrp = analyze_sentiment(xrp_summaries)
sentiment_results_lte = analyze_sentiment(lte_summaries)


data = {
    'Ticker': ['BTC'] * len(sentiment_results_btc) + ['ETH'] * len(sentiment_results_eth) +
              ['DOGE'] * len(sentiment_results_doge) + ['XRP'] * len(sentiment_results_xrp) +
              ['LTC'] * len(sentiment_results_lte),
    'Result_state': [result['label'] for result in sentiment_results_btc] +
                       [result['label'] for result in sentiment_results_eth] +
                       [result['label'] for result in sentiment_results_doge] +
                       [result['label'] for result in sentiment_results_xrp] +
                       [result['label'] for result in sentiment_results_lte],
    'Score': [result['score'] for result in sentiment_results_btc] +
                       [result['score'] for result in sentiment_results_eth] +
                       [result['score'] for result in sentiment_results_doge] +
                       [result['score'] for result in sentiment_results_xrp] +
                       [result['score'] for result in sentiment_results_lte],
}

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('results.csv', index=False)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [17]:
from google.colab import files
import math
from datetime import datetime, timedelta
coin_results = {}
coins = ['BTC', 'ETH', 'XRP', 'LTC', 'DOGE']
csv_file_path = "modified_data.csv"
coin_data = pd.read_csv(csv_file_path)
today = datetime.now()

tomorrow = today + timedelta(days=1)
for coin in coins:
    positive_sum = df[(df['Ticker'] == coin) & (df['Result_state'] == 'POSITIVE')]['Score'].sum()
    negative_sum = df[(df['Ticker'] == coin) & (df['Result_state'] == 'NEGATIVE')]['Score'].sum()
    if (len(df[(df['Ticker'] == coin)])) < 6:
         coin_results[coin] =  1-(((positive_sum - negative_sum)/(pow(len(df[(df['Ticker'] == coin)]),(1234/100)))))
    else:
        coin_results[coin] = 1-(((positive_sum - negative_sum)/(pow(len(df[(df['Ticker'] == coin)]),(13589029/10000000)))))
new_data = {
    "Date": [tomorrow.strftime('%Y-%m-%d')],
    "BTC": coin_results["BTC"],
    "ETH": coin_results["ETH"],
    "DOGE": coin_results["DOGE"],
    "LTC": coin_results["LTC"],
    "XRP": coin_results["XRP"],
}
new_data_df = pd.DataFrame(new_data)

coin_data = coin_data.append(new_data_df, ignore_index=True)
coin_data.to_csv(csv_file_path, index=False)

files.download('modified_data.csv')


<ipython-input-17-aa83ec8e3623>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_data = coin_data.append(new_data_df, ignore_index=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>